##Notebook 04 — Parameter Sweep & Policy Selection (LT / Close Factor / Bonus / Slippage)

This notebook turns the liquidation and bad-debt engine from Notebook 03 into a simple policy evaluation loop, sweeping over a small grid of protocol parameters to identify configurations that remain robust under benchmark stress conditions.

While Notebook 03 quantifies how bad debt forms for a single set of parameters, real protocol governance requires comparing many candidate settings. In practice, parameters such as liquidation threshold (LT), close factor, liquidation bonus, and execution slippage interact
nonlinearly: improving capital efficiency in normal conditions can increase tail losses, and stronger liquidation incentives can reduce bad debt at the cost of higher collateral extraction. This notebook systematically explores those trade-offs using a few representative
stress scenarios (mild/base/severe/crisis) as a lightweight “policy test suite.”

The workflow consists of:

	1.	Defining benchmark stress scenarios (price shock + delay proxy) to represent a small set of comparable market states for policy testing
	2.	Specifying parameter grids for LT, close factor, liquidation bonus,and slippage (kept intentionally small for fast iteration in Colab)
	3.	Running a parameter sweep by evaluating each policy configuration across all benchmark scenarios using the Notebook 03 liquidation engine
	4.	Aggregating results into policy tables, including scenario-specific bad debt rates and liquidatable rates for each configuration
	5.	Constructing a simple policy score (weighted toward severe/crisis) and ranking configurations to surface the most robust settings

The resulting ranked policy table provides an interpretable shortlist of candidate parameter configurations and serves as the foundation for later extensions such as constraint-based selection (e.g., cap maximum liquidatable rate), multi-objective scoring, and more granular scenario
coverage.

In [ ]:
#1. Imports

!rm -rf b1-defi-risk
!git clone https://github.com/lydialydia-lydia/b1-defi-risk.git
%cd b1-defi-risk

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.sim_data import generate_synthetic_positions
from src.liquidation import simulate_one_step_liquidation


Cloning into 'b1-defi-risk'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 58 (delta 21), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (58/58), 252.41 KiB | 2.02 MiB/s, done.
Resolving deltas: 100% (21/21), done.
/content/b1-defi-risk/b1-defi-risk


In [ ]:
#2. Generate positions
SEED = 42
positions = generate_synthetic_positions(seed=SEED)

price_init = float(positions["collateral_price_init"].iloc[0])
total_debt = float(positions["debt_amount"].sum())

print("positions:", positions.shape, "| price_init:", price_init)

positions: (10000, 8) | price_init: 2500.0


In [ ]:
#3. Stress scenarios (choose a few “benchmark” points)
# Pick benchmark stress points for policy comparison
benchmark_scenarios = [
    {"name": "mild",    "shock": -0.10, "delay": 0},  # small drawdown, no lag
    {"name": "base",    "shock": -0.20, "delay": 1},  # moderate drawdown, mild lag
    {"name": "severe",  "shock": -0.30, "delay": 2},  # large drawdown + stale oracle/execution
    {"name": "crisis",  "shock": -0.40, "delay": 3},  # tail event + worst lag proxy
]

# Simple proxy: delay → extra effective price deterioration
# (captures stale oracle / latency / slow liquidation queue effects)
delay_to_extra_drop = {0: 0.00, 1: -0.02, 2: -0.05, 3: -0.10}

In [ ]:
#4. Parameter grids
# Parameter sweep small grids, covers "loose vs tight" policy regimes
LT_grid = [0.75, 0.80, 0.85]    # liquidation threshold: lower = stricter (safer), higher = more capital efficient
close_factor_grid = [0.30, 0.50, 0.80]    # max debt repaid per liquidation event: higher = more aggressive cleanup
liq_bonus_grid = [0.05, 0.08, 0.12]   # incentive for liquidators: higher = faster liquidation, but costs more collateral
slippage_grid = [0.00, 0.02, 0.05]    # execution friction / price impact proxy: higher = worse fills / more bad debt risk

In [ ]:
#5. Run sweep
rows = []

for LT in LT_grid:
  for cf in close_factor_grid:
    for lb in liq_bonus_grid:
        for sl in slippage_grid:
          # Each (LT, cf, lb, sl) defines one "policy configuration"
          for s in benchmark_scenarios:
            total_shock = s["shock"] + delay_to_extra_drop[s["delay"]]    # Combine market shock + lag proxy into an effective total shock
            stressed_price = price_init * (1.0 + total_shock)

            sim = simulate_one_step_liquidation(
                collateral_amount=positions["collateral_amount"],
                debt_amount=positions["debt_amount"],
                price=stressed_price,
                liquidation_threshold=LT,
                close_factor=cf,
                liquidation_bonus=lb,
                slippage=sl
            )


            # liquidatable_rate: how many accounts cross HF<1 (trigger surface)
            # bad_debt_rate: residual loss after applying close_factor + frictions
            liquidatable_rate = sim["is_liquidatable"].mean()
            bad_debt_total = float(sim["bad_debt"].sum())
            bad_debt_rate = bad_debt_total / total_debt

            rows.append({
                "scenario": s["name"],
                "shock": s["shock"],
                "delay": s["delay"],
                "total_shock": total_shock,
                "LT": LT,
                "close_factor": cf,
                "liq_bonus": lb,
                "slippage": sl,
                "liquidatable_rate": liquidatable_rate,
                "bad_debt_rate": bad_debt_rate
            })

sweep = pd.DataFrame(rows)
sweep.head()

,scenario,shock,delay,total_shock,LT,close_factor,liq_bonus,slippage,liquidatable_rate,bad_debt_rate
0,mild,-0.1,0,-0.10,0.75,0.3,0.05,0.00,0.2525,0.208510
1,base,-0.2,1,-0.22,0.75,0.3,0.05,0.00,0.6751,0.508709
2,severe,-0.3,2,-0.35,0.75,0.3,0.05,0.00,0.9547,0.677284
3,crisis,-0.4,3,-0.50,0.75,0.3,0.05,0.00,0.9988,0.699410
4,mild,-0.1,0,-0.10,0.75,0.3,0.05,0.02,0.2525,0.208510


In [ ]:
#6. Build a “policy score” table (focus on severe+crisis)
# Pivot for scoring
pivot = sweep.pivot_table(
    index=["LT", "close_factor", "liq_bonus", "slippage"],
    columns="scenario",
    values="bad_debt_rate"
).add_suffix("_bad_debt_rate").reset_index().fillna(0.0)


pivot = pivot.fillna(0.0)

pivot["score"] = 0.7 * pivot["crisis_bad_debt_rate"]+ 0.3 * pivot["severe_bad_debt_rate"]

policy_rank = pivot.sort_values("score").reset_index(drop=True)
policy_rank.head(10)

scenario,LT,close_factor,liq_bonus,slippage,base_bad_debt_rate,crisis_bad_debt_rate,mild_bad_debt_rate,severe_bad_debt_rate,score
0,0.85,0.8,0.05,0.00,0.069600,0.259713,0.010408,0.167713,0.232113
1,0.80,0.8,0.05,0.00,0.107242,0.260467,0.028071,0.183791,0.237464
2,0.85,0.8,0.05,0.02,0.069601,0.269822,0.010408,0.168522,0.239432
3,0.75,0.8,0.05,0.00,0.145346,0.260700,0.059574,0.194650,0.240885
4,0.85,0.8,0.08,0.00,0.069601,0.273958,0.010408,0.168935,0.242451
5,0.80,0.8,0.05,0.02,0.107243,0.270576,0.028071,0.184600,0.244783
6,0.80,0.8,0.08,0.00,0.107244,0.274712,0.028071,0.185013,0.247802
7,0.75,0.8,0.05,0.02,0.145347,0.270809,0.059574,0.195459,0.248204
8,0.85,0.8,0.08,0.02,0.069602,0.284734,0.010408,0.170235,0.250384
9,0.75,0.8,0.08,0.00,0.145348,0.274945,0.059574,0.195872,0.251223


In [ ]:
#7. Policy rank
pivot = sweep.pivot_table(
    index=["LT","close_factor","liq_bonus","slippage"],
    columns="scenario",
    values="bad_debt_rate"
).add_suffix("_bad_debt_rate").reset_index().fillna(0.0)

pivot["score"] = 0.7 * pivot["crisis_bad_debt_rate"] + 0.3 * pivot["severe_bad_debt_rate"]
policy_rank = pivot.sort_values("score").reset_index(drop=True)

In [ ]:
#8. Show top configs
# Liquidation-rate pivot (add suffix to avoid name collisions)
liq_pivot = sweep.pivot_table(
    index=["LT", "close_factor", "liq_bonus", "slippage"],
    columns="scenario",
    values="liquidatable_rate"
).add_suffix("_liquidatable_rate").reset_index().fillna(0.0)

topN = 10
top = policy_rank.head(topN).merge(
    liq_pivot,
    on=["LT", "close_factor", "liq_bonus", "slippage"],
    how="left"
).fillna(0.0)

# Display key columns (now names are deterministic)
top[[
    "LT","close_factor","liq_bonus","slippage","score",
    "severe_bad_debt_rate","crisis_bad_debt_rate",
    "severe_liquidatable_rate","crisis_liquidatable_rate"
]]

scenario,LT,close_factor,liq_bonus,slippage,score,severe_bad_debt_rate,crisis_bad_debt_rate,severe_liquidatable_rate,crisis_liquidatable_rate
0,0.85,0.8,0.05,0.00,0.232113,0.167713,0.259713,0.8067,0.9933
1,0.80,0.8,0.05,0.00,0.237464,0.183791,0.260467,0.8964,0.9972
2,0.85,0.8,0.05,0.02,0.239432,0.168522,0.269822,0.8067,0.9933
3,0.75,0.8,0.05,0.00,0.240885,0.194650,0.260700,0.9547,0.9988
4,0.85,0.8,0.08,0.00,0.242451,0.168935,0.273958,0.8067,0.9933
5,0.80,0.8,0.05,0.02,0.244783,0.184600,0.270576,0.8964,0.9972
6,0.80,0.8,0.08,0.00,0.247802,0.185013,0.274712,0.8964,0.9972
7,0.75,0.8,0.05,0.02,0.248204,0.195459,0.270809,0.9547,0.9988
8,0.85,0.8,0.08,0.02,0.250384,0.170235,0.284734,0.8067,0.9933
9,0.75,0.8,0.08,0.00,0.251223,0.195872,0.274945,0.9547,0.9988


In [ ]:
#Save tables
from pathlib import Path

OUT_DIR = Path("/content/b1-defi-risk/outputs/tables")
OUT_DIR.mkdir(parents=True, exist_ok=True)

sweep_path = OUT_DIR / "04_sweep_raw.csv"
policy_rank_path = OUT_DIR / "04_policy_rank.csv"
top_path = OUT_DIR / "04_top10_policies.csv"

sweep.to_csv(sweep_path, index=False)
policy_rank.to_csv(policy_rank_path, index=False)
top.to_csv(top_path, index=False)

print("Saved:")
print("-", sweep_path)
print("-", policy_rank_path)
print("-", top_path)

Saved:
- /content/b1-defi-risk/outputs/tables/04_sweep_raw.csv
- /content/b1-defi-risk/outputs/tables/04_policy_rank.csv
- /content/b1-defi-risk/outputs/tables/04_top10_policies.csv
